In [ ]:
import numpy as np
import importlib, os, datetime
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter


from quick_sim import setup_sim


# import edward_tools.fq_runner as fq_runner
from edward_tools.simple_1D_potential import flux_qubit_1D_non_linear_approx_pot, flux_qubit_1D_non_linear_approx_force
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation
from edward_tools import visualization

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 4.2
# T = 7
k_BT = k_B * T

C_factor = 1
L_factor = 1
R_factor = 1000
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0


I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 1 * R_factor, 1 * R_factor                                         # ohm
C_1, C_2 = 500e-15 * C_factor, 500e-15 * C_factor                             # F

L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
L_1, L_2 = 5e-12 * L_factor, 5e-12 * L_factor                             # H 
freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)
print(f"freq = {freq / 1e9:.3f}GHz, characteristic_time  = {characteristic_time * 1e12:.3f}ps")


In [ ]:

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

beta_1 = 2.3
beta_2 = 2.3

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;

_lambda = np.array([lambda_2])
_theta  = np.array([theta_2])
_eta  =   np.array([eta_2])

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 10
params['dt'] = 1/500
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_2]
params['mass'] = np.array([1])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::500] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_off,  "phi_2_x": phi_2_x_off,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": 1, 'x_c': x_c
}


In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_2_bound], [phi_2_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(flux_qubit_1D_non_linear_approx_pot, flux_qubit_1D_non_linear_approx_force, 14, 1, \
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

# coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot_special, coupled_flux_qubit_non_linear_approx_force_special, 14, 4,\
                           # default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

In [ ]:
zeroDissipation = False
# zeroDissipation = True

In [ ]:
saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "characteristic_time": np.sqrt(C_1 * C_factor * L_1 * L_factor),
    "phi_1_x_on": phi_1_x_on_12, "phi_2_x_on": phi_2_x_on_12,
    "phi_1_dcx_on": phi_1_dcx_on_12, "phi_2_dcx_on": phi_2_dcx_on_12, "M_12_on": M_12_on,
    "gamma": gamma
}


# bookmark
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
phi_1_dcx_off, phi_2_dcx_off, M_12_off


initial_parameter_dict["phi_1_dcx"] = phi_1_dcx_off
initial_parameter_dict["phi_2_dcx"] = phi_2_dcx_off
initial_parameter_dict["phi_1_x"]   = phi_1_x_off
initial_parameter_dict["phi_2_x"]   = phi_2_x_off
initial_parameter_dict["M_12"]      = M_12_off

In [ ]:

# longer CE
t_duration = 100

ratio_array = [0.998] * 5
# ratio_array = [0.9, 0.9, 0.9, 0.9, 0.9]
protocol_list_variable_duration = [
    # forward
    
    # create_CE_Protocol(50, CE_break_1),
    # no change
    # create_CE_Protocol(100, {
    # "phi_1_x": 0, "phi_2_x": 0, "M_12": initial_parameter_dict["M_12"], \
    # "phi_1_dcx": 0, "phi_2_dcx": 0, "name":"no change"
    # }),
    
    # decrease KE
    # create_CE_Protocol(100, {
    # "phi_1_x": 0, "phi_2_x": 0, "M_12": 0.1, \
    # "phi_1_dcx": 0, "phi_2_dcx": 0, "name":"decrease k"
    # }),
    
    # create_CE_Protocol(50, {
    # "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
    # "phi_1_dcx": 0, "phi_2_dcx": 1.7, "name":"decrease k"
    # }),
    create_CE_Protocol(150, {
    "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
    "phi_1_dcx": 0, "phi_2_dcx": 1.7, "name":"decrease k"
    }),
    create_CE_Protocol(150, {
    "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
    "phi_1_dcx": 0, "phi_2_dcx": 0, "name":"decrease k"
    }),
    # create_CE_Protocol(100, {
    # "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
    # "phi_1_dcx": 0, "phi_2_dcx": 0, "name":"decrease k"
    # }),

    
    # increase KE    
    # create_CE_Protocol(100, {
    # "phi_1_x": 0, "phi_2_x": 0, "M_12": 10, \
    # "phi_1_dcx": 0, "phi_2_dcx": 0, "name":"increasing k"
    # })    
]

protocol_list = protocol_list_variable_duration

In [ ]:
# protocol_list_variable_duration = [
#     # forward
#     create_CE_Protocol(10, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 0, "name":"decrease k"
#     }),
#     create_CE_Protocol(20, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 0.5, "name":"decrease k"
#     }),
#     create_CE_Protocol(30, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 1.0, "name":"decrease k"
#     }),
#    create_CE_Protocol(40, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 1.5, "name":"decrease k"
#     }),
#    create_CE_Protocol(50, {
#     "phi_1_x": 0, "phi_2_x": 0, "M_12": 0, \
#     "phi_1_dcx": 0, "phi_2_dcx": 2.0, "name":"decrease k"
#     })
#     ]
# protocol_list = protocol_list_variable_duration

In [ ]:
params.items()

# setting initial states

In [ ]:
%%capture


regenerate_init_state = False
regenerate_init_state = True
if regenerate_init_state:
    initial_potential_protocol_list = [create_CE_Protocol(10, {'phi_1_x': 0, 'phi_2_x': 0, 'M_12': 0, 'phi_1_dcx': 0, 'phi_2_dcx': 0, 'name': 'test'})]
    init_state_saved = cfq_batch_sweep.create_initial_state(initial_parameter_dict, initial_potential_protocol_list, coupled_fq_pot, params)
else:
    init_state_saved = np.load("four_well_default_init_state_N_1000.npy")
    pass

init_state_used = init_state_saved

In [ ]:
# customize the initial state
init_state_saved[:, 0, 0] = np.random.normal(0.3, 0.1, params['N'])
init_state_saved[:, 0, 1] = np.random.normal(0, 0.01, params['N'])

# create cfqr object

In [ ]:

computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(init_state=init_state_saved)
cfqr.set_sim_attributes(init_state=init_state_used)


In [ ]:

time_array = np.linspace(cfqr.protocol_time_array[0], cfqr.protocol_time_array[-1], 5)
phi_2_array = np.linspace(-4, 4, 100)
fig, ax = plt.subplots(1, 3, figsize = [15, 5])

for _t in time_array[:3]:
    params_at_t = cfqr.convert_params_list_into_dict(cfqr.protocol.get_params(_t))
    U, X = cfqr.system.lattice(_t, resolution = 100)

    _phi_2_xdc = params_at_t['phi_2_dcx']    
    ax[0].plot(X[0], U[0], label = f"t = {_t:.3g}, phi_2xdc = {params_at_t['phi_2_dcx']:.3f}")
    ax[0].set_title(r"potential vs $\varphi_2$")
    ax[0].set_xlabel(r"$\varphi_2$")
    ax[0].set_ylabel(r"potential ($k_BT$)")
    ax[0].set_ylim(250, 1300)
    
    slope_array = [_phi_2 - beta_2 * np.sin(_phi_2) * np.cos(_phi_2_xdc/2) for _phi_2 in phi_2_array]
    ax[1].plot(phi_2_array, slope_array, label = f"t = {_t:.3g}, phi_2xdc = {params_at_t['phi_2_dcx']:.3f}")
    ax[1].set_title(r"slope vs $\varphi_2$")
    ax[1].set_xlabel(r"$\varphi_2$")
    ax[1].set_ylabel(r"slope")
    
    slope_array = [_phi_2 - beta_2 * np.sin(_phi_2) * np.cos(_phi_2_xdc/2) for _phi_2 in phi_2_array]
    ax[2].plot(phi_2_array, np.abs(slope_array), label = f"t = {_t:.3g}, phi_2xdc = {params_at_t['phi_2_dcx']:.3f}")
    ax[2].set_title(r"|slope| vs $\varphi_2$")
    ax[2].set_xlabel(r"$\varphi_2$")
    ax[2].set_ylabel(r"|slope|")

    # curvature_array = [1-beta_2 * np.cos(_phi_2) * np.cos(_phi_2_xdc/2) for _phi_2 in phi_2_array]
    # ax[2].plot(phi_2_array, curvature_array, label = f"t = {_t:.3g}, phi_2xdc = {params_at_t['phi_2_dcx']:.3f}")
    # ax[2].set_title(r"Curvature vs $\varphi_2$")
    # ax[2].set_xlabel(r"$\varphi_2$")
    # ax[2].set_ylabel(r"curvature")

ax[0].legend()

# Simulation

In [ ]:
importlib.reload(cfq_batch_sweep)

manual_domain=[np.array([-5, -5]), np.array([5, 5])]

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = init_state_used, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, measure_all_states=True, \
                        has_velocity = has_velocity, fidelityTest = False)
print(f"characteristic_time = {characteristic_time * 10e9:.3g}ns")

# analysis

In [ ]:
U, X = simResult['cfqr'].system.lattice(t=0, resolution = 50)

In [ ]:
# get particle's potential and position
all_states = simResult['cfqr'].get_all_state()
U_init = simResult['cfqr'].system.get_potential( simResult['cfqr'].init_state, 0)
KE_init = simResult['cfqr'].system.get_kinetic_energy( simResult['cfqr'].init_state)

x_of_all_particles = all_states[:, :, 0, 0]
simulation_time_array = simResult['cfqr'].protocol_all_time_array[params['as_step']]

In [ ]:
print(f"average KE = {np.mean(KE_init):.3f}")

In [ ]:
i = 2

U_of_all_particles_init = simResult['cfqr'].system.get_potential(all_states[:, 0, ...] , 0)
x_of_all_particles_init = x_of_all_particles[:, 0, ...]

U_of_all_particles_at_t_i = simResult['cfqr'].system.get_potential( all_states[:, i, ...], i)
x_of_all_particles_at_t_i = x_of_all_particles[:, i, ...]


time_step_array = simResult['cfqr'].protocol_all_time_array[params['as_step']]
final_KE = simResult['cfqr'].system.get_kinetic_energy(all_states[:, -1, ...])
print(f"Final KE: {np.mean(final_KE)}")


In [ ]:
particle_index = 0
KE_of_particle = simResult['cfqr'].system.get_kinetic_energy(all_states[particle_index, ...])
PE_of_particle = [simResult['cfqr'].system.get_potential(all_states[particle_index, _i, ...], _t) for _i, _t in enumerate(time_step_array)]
TE_of_particle = KE_of_particle + PE_of_particle

WD_on_particle = np.array(simResult['cfqr'].sim.work_dist_time_array).T[particle_index][params["as_step"]]
WD_on_particle = np.concatenate([np.array([0]), WD_on_particle]) # add 0 to the WD array
v_of_particle = all_states[particle_index, :, 0, 1]


plt.plot(time_step_array, np.array(KE_of_particle), label = "KE")
plt.plot(time_step_array, np.array(PE_of_particle), label = "PE")
plt.plot(time_step_array, WD_on_particle, label = "WD")
# plt.plot(time_step_array, np.array(TE_of_particle), label = "TE")

plt.legend()
plt.show()

print(f"maximum work done: {WD_on_particle[-1]}")



In [ ]:
# randomly chose 10}}

In [ ]:
# a = np.arange(10)
# np.random.shuffle(a)
# particles_chosen = a[0:10]

# fig, ax = plt.subplots(2, 5, figsize = [15, 8])
# ax_flatten = ax.flatten()
# for ax_index, particle_index in enumerate(particles_chosen):
#     KE_of_particle = simResult['cfqr'].system.get_kinetic_energy(all_states[particle_index, ...])
#     PE_of_particle = [simResult['cfqr'].system.get_potential(all_states[particle_index, _i, ...], _t) for _i, _t in enumerate(time_step_array)]
#     WD_on_particle = np.array(simResult['cfqr'].sim.work_dist_time_array).T[particle_index][params["as_step"]]
#     WD_on_particle = np.concatenate([np.array([0]), WD_on_particle]) # add 0 to the WD array

#     # ax_flatten[ax_index].plot(time_step_array, KE_of_particle + np.array(PE_of_particle), label = "TE")
#     ax_flatten[ax_index].plot(time_step_array, np.array(KE_of_particle), label = "KE")
#     ax_flatten[ax_index].plot(time_step_array, np.array(PE_of_particle), label = "PE")
#     # ax_flatten[ax_index].plot(time_step_array, WD_on_particle, label = "WD")
#     # ax_flatten[ax_index].set_ylim(-2, 5)
#     if ax_index == 0:
#         ax_flatten[ax_index].legend()

In [ ]:
particle_index = 0
def animate_sim_flux_qubit_with_cutline_and_projection(cfqr, time_array=None, legend=True, 
    plot_axis = [0, 1], fig_ax=None, PE_graph_setting = {"xlim": [-4, 4], "ylim": [-2, 300]}, particle_index = None, **pot_kwargs):
    """
    This is very similar to the function animate_sim_flux_qubit_with_cutline, but it can project particles onto the cutlines
    """
    
    names = [r"$\varphi_1$", r"$\varphi_2$", r"$\varphi_{1dc}$", r"$\varphi_{2dc}$"]
    
    if not fig_ax:
        fig, ax = plt.subplots(1, 2, figsize=[10, 5])
    else:
        fig, ax = fig_ax
    ax_flatten = ax.flatten()

    # index_skip_in_all_time_array = int(1/simResult['cfqr'].params['dt'])
    # as_step_adjust = params['as_step'].start, params['as_step'].stop, params['as_step'].step * 2
    time_array = cfqr.protocol_all_time_array[params['as_step']]
    parms_at_init = cfqr.protocol.get_params(0)
    plot_axis = [0, 1]
    

    params_at_all_time = np.array([cfqr.protocol.get_params(_t) for _t in  cfqr.protocol_all_time_array]).T
    params_at_all_time_dict = dict(zip(cfqr.protocol_key, params_at_all_time))
    
    
    U, X = cfqr.system.lattice(t=0, resolution = 50) 
    U, X = U[0], X[0]
    l = ax_flatten[0].plot(X, U)[0]
    ax_flatten[0].set_xlim(PE_graph_setting['xlim'])
    ax_flatten[0].set_ylim(PE_graph_setting['ylim'])
    
    all_states = simResult['cfqr'].get_all_state()
    
    
    # plot the energy profile graph
    p_index = 0
    KE_of_particle = simResult['cfqr'].system.get_kinetic_energy(all_states[p_index, ...])
    PE_of_particle = [simResult['cfqr'].system.get_potential(all_states[p_index, _i, ...], _t) for _i, _t in enumerate(time_step_array)]
    TE_of_particle = KE_of_particle + PE_of_particle

    WD_on_particle = np.array(simResult['cfqr'].sim.work_dist_time_array).T[p_index][simResult['cfqr'].params["as_step"]]
    WD_on_particle = np.concatenate([np.array([0]), WD_on_particle]) # add 0 to the WD array
    v_of_particle = all_states[p_index, :, 0, 1]

    ax_flatten[1].set_title(f"energy profile of particle {p_index}")
    ax_flatten[1].plot(time_step_array, np.array(KE_of_particle), label = "KE")
    ax_flatten[1].plot(time_step_array, np.array(PE_of_particle), label = "PE")
    ax_flatten[1].plot(time_step_array, WD_on_particle, label = "WD")
    ax_flatten[1].plot(time_step_array, np.array(TE_of_particle), label = "TE")
    ax_flatten[1].legend()
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    xmax, ymax = plt.xlim()[1], plt.ylim()[1]
    energy_description = f" max WD: {WD_on_particle[-1]:.3g},\n max KE = {np.max(KE_of_particle):.3g}" +  \
                    f",\n max PE = {np.max(PE_of_particle):.3g}"
    ax_flatten[1].text(int(xmax * 0.4), int(ymax * 0.85), energy_description, fontsize=11, verticalalignment='top', 
        bbox=props)
    
    
    # animation playing
    if particle_index != None:
        x_of_particle = all_states[particle_index, :, 0, 0]
        
        U_of_particle_init = simResult['cfqr'].system.get_potential(all_states[particle_index, 0, ...] , 0)
        x_of_particle_init = x_of_particle[0, ...]
        
        KE_of_particle = simResult['cfqr'].system.get_kinetic_energy(all_states[particle_index, ...])
        PE_of_particle = [simResult['cfqr'].system.get_potential(all_states[particle_index, _i, ...], _t) for _i, _t in enumerate(time_step_array)]
        TE_of_particle = KE_of_particle + PE_of_particle

        WD_on_particle = np.array(simResult['cfqr'].sim.work_dist_time_array).T[particle_index][simResult['cfqr'].params["as_step"]]
        WD_on_particle = np.concatenate([np.array([0]), WD_on_particle]) # add 0 to the WD array
        v_of_particle = all_states[particle_index, :, 0, 1]

        
        ax_flatten[1].plot(time_step_array, np.array(KE_of_particle), label = "KE")
        ax_flatten[1].plot(time_step_array, np.array(PE_of_particle), label = "PE")
        ax_flatten[1].plot(time_step_array, WD_on_particle, label = "WD")
        ax_flatten[1].plot(time_step_array, np.array(TE_of_particle), label = "TE")
        
    
    else:
        x_of_particle = all_states[:, :, 0, 0]
        U_of_particle_init = simResult['cfqr'].system.get_potential(all_states[:, 0, ...] , 0)
        x_of_particle_init = x_of_particle[:, 0, ...]
    
    scatter_plot = ax[0].scatter(x_of_particle_init, U_of_particle_init)
    
    def animate(i):
        parms_at_i = cfqr.protocol.get_params(time_array[i])
        # ax_flatten[1].clear()
        
        U, X = cfqr.system.lattice(t=time_array[i], resolution = 50) 
        U, X = U[0], X[0]
        l.set_data(X, U)
        
        if particle_index != None:
            U_of_particle_at_t_i = simResult['cfqr'].system.get_potential( all_states[particle_index, i, ...], time_array[i])
            x_of_particle_at_t_i = x_of_particle[i, ...]


        else:
            U_of_particle_at_t_i = simResult['cfqr'].system.get_potential( all_states[:, i, ...], time_array[i])
            x_of_particle_at_t_i = x_of_particle[:, i, ...]
            ax_flatten[0].set_title(f"t = {time_array[i]}/{time_array[-1]}")            
        scatter_plot.set_offsets(np.c_[x_of_particle_at_t_i, U_of_particle_at_t_i])
        
    ani = FuncAnimation(fig, animate, interval=50, frames=len(time_array), blit=False)
    return ani, fig, ax

In [ ]:
PE_graph_setting = {"xlim": [-4, 4], "ylim": [350, 1000]}
ani, fig, ax = animate_sim_flux_qubit_with_cutline_and_projection(simResult['cfqr'], PE_graph_setting =  PE_graph_setting)
fig.tight_layout()
writer = animation.PillowWriter(fps=100, metadata=dict(artist='Me'), bitrate=1800)
ani.save('scatter.gif', writer=writer)
plt.close()

In [ ]:
Image(filename="scatter.gif")

In [ ]:
plt.hist(simResult['cfqr'].sim.work_dist_array, bins = 50)
plt.show()

In [ ]:
_R = [100, 500, 1000, 5000, 10000] 	
_WD = [43.867198364159734, 53.727594202060956, 84.59186846712807, 148.50721839697968, 155.30489729163008]
plt.scatter(_R, _WD)
# plt.xscale('log')
plt.xlabel("Resistance (Ω)")
plt.ylabel("extractable work (k_BT)")